In [1]:
!pip install transformers

In [2]:
!pip install tensorflow transformers keras

In [3]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 85.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=9a7d2f8033a516008aaffde4d7706774a352aeb421f85c2afa98c0ee13ccad78
  Stored in directory: /root/.cache/pip/wheels/6a/67/f8/9cf1a8ff87e0b37f738769df49cc142a655489a6d27b68089f
Successfully built langdetect


In [4]:
!pip install wordcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.4/460.4 kB 55.3 MB/s eta 0:00:00


In [5]:
!pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 17.8 MB/s eta 0:00:00


In [6]:
!pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 115.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 76.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 28.6 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 18.2.0
    Uninstalling attrs-18.2.0:
      Successfully uninstalled attrs-18.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.


In [7]:
# Importing libraries
import math

import numpy as np
import pandas as pd
import torch

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from tqdm.auto import tqdm

import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import spacy
import random

from langdetect import detect
from wordcloud import WordCloud

from keras.layers import Input, Dense
from keras.models import Model

from geopy.geocoders import Nominatim
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import BertForSequenceClassification, BertModel, BertTokenizer, TFBertModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import DataLoader, Dataset

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords as stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
df = pd.read_csv('merged_cluster100.csv')


In [9]:
df


,id,text,user_id,timestamp,cluster_id,lat,lng,language_label,year,month,day,hour,minute,second,cluster_label,distance_from_center
0,0,wow ’ know homies coming . lit .,5.853807e+07,2021-08-02 01:25:46,2,34.020789,-118.411907,0,2021.0,8.0,2.0,1.0,25.0,46.0,1,13.970579
1,1,foggy nights peaceful calming chill vibe,1.032647e+18,2021-08-28 08:47:36,2,34.020789,-118.411907,0,2021.0,8.0,28.0,8.0,47.0,36.0,1,13.970579
2,2,khamisvideobomb,2.781174e+07,2021-08-13 15:49:15,2,34.020789,-118.411907,24,2021.0,8.0,13.0,15.0,49.0,15.0,1,13.970579
3,3,anyone tryna hit gym 4 hmu,5.490040e+08,2021-08-31 22:41:20,2,34.020789,-118.411907,24,2021.0,8.0,31.0,22.0,41.0,20.0,1,13.970579
4,4,kidding intro crazy .,2.963951e+07,2021-08-06 03:44:29,2,34.020789,-118.411907,4,2021.0,8.0,6.0,3.0,44.0,29.0,1,13.970579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604201,604201,students must dressed appropriately day arrive...,1.990241e+08,2021-08-12 11:08:07,2996,29.668301,-95.158936,0,2021.0,8.0,12.0,11.0,8.0,7.0,4,33.786445
604202,604202,actually hate working lunch shift crew job,1.382980e+18,2021-08-04 15:35:10,2996,29.668301,-95.158936,0,2021.0,8.0,4.0,15.0,35.0,10.0,4,33.786445
604203,604203,’ reading thisyo mama,1.039173e+18,2021-08-24 00:49:46,2996,29.668301,-95.158936,24,2021.0,8.0,24.0,0.0,49.0,46.0,4,33.786445
604204,604204,mean secondddd,1.046619e+18,2021-08-02 17:02:13,2996,29.668301,-95.158936,0,2021.0,8.0,2.0,17.0,2.0,13.0,4,33.786445


In [10]:
# # Take a random sample size of merged dataset
# sample_size = 20000
# df = df.sample(n=sample_size)
# df


In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [12]:
# Step 1: Prepare your data
# Assuming you have your data stored in a DataFrame called 'df'
df['text'] = df['text'].fillna(' ')
text_data = df['text'].tolist()
location_labels = df[['lat', 'lng']].values


In [13]:
# Step 3: Load DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', normalize=True)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

In [14]:
# Step 4: Prepare the model for prediction
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [15]:
# Step 5: Split your data
train_texts, test_texts, train_labels, test_labels = train_test_split(text_data, location_labels, test_size=0.3, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.3, random_state=42)


In [16]:
# Normalize location labels
scaler = MinMaxScaler()
train_labels_normalized = scaler.fit_transform(train_labels)
val_labels_normalized = scaler.transform(val_labels)
test_labels_normalized = scaler.transform(test_labels)


In [17]:
# Step 6: Train the model
class LocationDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        return {
            'input_ids': input_ids.to(device),
            'attention_mask': attention_mask.to(device),
            'label': torch.tensor(label, dtype=torch.float).to(device)
        }

train_dataset = LocationDataset(train_texts, train_labels_normalized)
val_dataset = LocationDataset(val_texts, val_labels_normalized)

batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

optimizer = AdamW(model.parameters(), lr=2e-4)  # Adjust the learning rate here
loss_fn = torch.nn.MSELoss()

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits.squeeze()
        loss = loss_fn(logits, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = train_loss / len(train_dataloader)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['label']
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits.squeeze()
            loss = loss_fn(logits, labels)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_dataloader)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f} - Val Loss: {avg_val_loss:.4f}")


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5 - Train Loss: 0.0205 - Val Loss: 0.0193
Epoch 2/5 - Train Loss: 0.0193 - Val Loss: 0.0193
Epoch 3/5 - Train Loss: 0.0186 - Val Loss: 0.0195
Epoch 4/5 - Train Loss: 0.0179 - Val Loss: 0.0196
Epoch 5/5 - Train Loss: 0.0171 - Val Loss: 0.0196


In [18]:
# Step 7: Evaluate the model on the test set
test_dataset = LocationDataset(test_texts, test_labels_normalized)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()
test_loss = 0.0
predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits.squeeze()
        loss = loss_fn(logits, labels)
        test_loss += loss.item()
        # Convert logits to predicted labels
        predicted_labels = logits.detach().cpu().numpy()
        predictions.extend(predicted_labels)

avg_test_loss = test_loss / len(test_dataloader)
print(f"Average Test Loss: {avg_test_loss}")

# Convert predictions and labels back to original scale
predictions = scaler.inverse_transform(predictions)
test_labels = scaler.inverse_transform(test_labels_normalized)

# Calculate evaluation metrics
mse = mean_squared_error(test_labels, predictions)
mae = mean_absolute_error(test_labels, predictions)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")


Average Test Loss: 0.019488021182379578
Mean Squared Error (MSE): 144.5557037549921
Mean Absolute Error (MAE): 8.688388651353918


In [19]:
# # Step 6: Train the model
# class LocationDataset(Dataset):
#     def __init__(self, texts, labels):
#         self.texts = texts
#         self.labels = labels
    
#     def __len__(self):
#         return len(self.texts)
    
#     def __getitem__(self, idx):
#         text = self.texts[idx]
#         label = self.labels[idx]
#         encoding = tokenizer.encode_plus(
#             text,
#             add_special_tokens=True,
#             max_length=512,
#             padding='max_length',
#             truncation=True,
#             return_tensors='pt'
#         )
#         input_ids = encoding['input_ids'].squeeze()
#         attention_mask = encoding['attention_mask'].squeeze()
#         return {
#             'input_ids': input_ids.to(device),
#             'attention_mask': attention_mask.to(device),
#             'label': torch.tensor(label, dtype=torch.float).to(device)
#         }

# train_dataset = LocationDataset(train_texts, train_labels)
# val_dataset = LocationDataset(val_texts, val_labels)
# test_dataset = LocationDataset(test_texts, test_labels)

# batch_size = 8
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# optimizer = AdamW(model.parameters(), lr=1e-4)
# loss_fn = torch.nn.MSELoss()

# num_epochs = 5
# for epoch in range(num_epochs):
#     model.train()
#     train_loss = 0.0
#     for batch in train_dataloader:
#         optimizer.zero_grad()
#         input_ids = batch['input_ids']
#         attention_mask = batch['attention_mask']
#         labels = batch['label']
#         outputs = model(input_ids, attention_mask=attention_mask)
#         logits = outputs.logits.squeeze()
#         loss = loss_fn(logits, labels)
#         train_loss += loss.item()
#         loss.backward()
#         optimizer.step()
#     avg_train_loss = train_loss / len(train_dataloader)
#     print(f"Epoch {epoch+1}/{num_epochs} - Average Training Loss: {avg_train_loss}")
          

In [20]:
# # Step 7: Evaluate the model
# model.eval()
# test_loss = 0.0
# predictions = []
# with torch.no_grad():
#     for batch in test_dataloader:
#         input_ids = batch['input_ids']
#         attention_mask = batch['attention_mask']
#         labels = batch['label']
#         outputs = model(input_ids, attention_mask=attention_mask)
#         logits = outputs.logits.squeeze()
#         loss = loss_fn(logits, labels)
#         test_loss += loss.item()
#         # Convert logits to predicted labels
#         predicted_labels = logits.detach().cpu().numpy()
#         predictions.extend(predicted_labels)

# avg_test_loss = test_loss / len(test_dataloader)
# print(f"Average Test Loss: {avg_test_loss}")

# # Calculate accuracy
# predictions = np.array(predictions)
# predicted_locations = predictions.round()  # Round the predicted values to the nearest integers
# test_locations = test_labels  # Assuming you have the true labels stored in 'test_labels'

# # Compute accuracy
# accuracy = (predicted_locations == test_locations).mean()
# print(f"Accuracy: {accuracy}")
